In [1]:
import os 
import sys 
sys.path.append('../lznestpy')

import nestpy
from random import choices
from nestUtils import *

import matplotlib.pyplot as plt
import numpy as np

from LXeSimulation import *



In [2]:
def simulate_LXe_NEST(EventRateFile, nEvents, g1, g2):
    trueEventRate = np.loadtxt(EventRateFile, skiprows=0,delimiter=',')
    if trueEventRate.shape[0] >= 1500:
        trueEventRate = trueEventRate[:1000,:]
    reco_energies, S1s, spikes, S2s, weights = simulate_LXe(EventRateFile, nEvents, g1, g2)
    cutMask = (S1s >= 3) & (S1s <= 80) & (spikes >= 3)
    return reco_energies[cutMask], weights[cutMask], S1s[cutMask], S2s[cutMask]


def significance(signalRate, bkgRate, exposure):
    return signalRate * exposure / np.sqrt(bkgRate*exposure)


def findXenonSingalRate(eventRateFile, g1, g2):
    eventRate = np.loadtxt(eventRateFile, skiprows=0,delimiter=',')
    energy, weight, S1, S2 = simulate_LXe_NEST(eventRateFile, 1000000, g1, g2)
    hist, edges = np.histogram(energy, bins=np.arange(0, 100), weights=0.9*weight)
    total_rate = np.sum(hist)
    return total_rate, hist, edges


def findXenonModelSignificance(eventRateFile, bkgFile, g1=0.114, g2=47.1, exposure=15.3):
    total_rate, hist, edges = findXenonSingalRate(eventRateFile, g1, g2)
    bkgRates = np.loadtxt(bkgFile, skiprows=0,delimiter=',')
    bkgRate = np.trapz(bkgRates[:,1], bkgRates[:,0])/0.9
    return significance(total_rate, bkgRate*0.005, exposure)










In [5]:
import os as os


LZSR1g1, LZSR1g2, LZExposure = 0.114, 47.1, 15.3

xenonSignificanceTable = []
xenonSignificanceTable_unit_exposure = []

XeEventRateFiles = sorted(os.listdir("/Users/yxu/workbenches/XeArLoopworkbench/EventRatesMMA/Xenon"))

for XeEventRateFile in XeEventRateFiles:
    significance_current = findXenonModelSignificance(f"/Users/yxu/workbenches/XeArLoopworkbench/EventRatesMMA/Xenon/{XeEventRateFile}", 
                                                              f"/Users/yxu/workbenches/XeArLoopworkbench/LZSR1_bkg_total.txt", 
                                                              LZSR1g1, LZSR1g2, LZExposure)    
    significance_current_unit_exposure = findXenonModelSignificance(f"/Users/yxu/workbenches/XeArLoopworkbench/EventRatesMMA/Xenon/{XeEventRateFile}", 
                                                              f"/Users/yxu/workbenches/XeArLoopworkbench/LZSR1_bkg_total.txt", 
                                                              LZSR1g1, LZSR1g2, 1)   
     
    xenonSignificanceTable.append(significance_current)
    xenonSignificanceTable_unit_exposure.append(significance_current_unit_exposure)
    print(XeEventRateFile, significance_current)


XXe10Lightp1Table.csv 5.674779833248668e-08
XXe10Lightp1lTable.csv 5.6584175480412304e-08
XXe10Lightp2Table.csv 0.0030724310303374964
XXe10Lightp2Tlable.csv 0.005329759481808506
XXe10LightpTable.csv 0.003044731548651097
XXe10LightplTable.csv 0.0053642480841466535
XXe10Lightz1Table.csv 0.018111275954825937
XXe10Lightz1lTable.csv 0.01813529621471298
XXe10Lightz2Table.csv 0.05040718255610657
XXe10Lightz2lTable.csv 0.10278716723185079
XXe10LightzTable.csv 0.06526927714189291
XXe10LightzlTable.csv 0.1179210101335847
XXe10p1Table.csv 2.685502032029975e-09
XXe10p1lTable.csv 2.6828357531187944e-09
XXe10p2Table.csv 0.0004231452547394053
XXe10p2lable.csv 0.0007442283557088363
XXe10pTable.csv 0.00042261794649978076
XXe10plTable.csv 0.0007438456338026581
XXe10z1Table.csv 0.0008585403444238849
XXe10z1lTable.csv 0.0008602815801271702
XXe10z2Table.csv 0.006261993598166455
XXe10z2lTable.csv 0.011418151268775326
XXe10zTable.csv 0.006935665147316502
XXe10zlTable.csv 0.012238394376516696
XXe200Lightp1Tab

In [7]:
%pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [7]:
def parse_xenon_filename_auto(filename):
    """
    Parse Argon filename to extract model parameters
    Format example: mpi7_PS-PS_mchi10_gchi0.6_cHiggs0_EventRate
    
    Returns:
        mphi: Mediator mass in GeV (float)
        is_ps_ps: Whether interaction type is PS-PS (bool)
        mchi: DM mass in GeV (float) 
        gchi: DM coupling (float)
        c_higgs: Higgs coupling (float)
    """
    import re
    
    # Extract mphi
    mphi = float(re.search(r'mpi(\d+(?:\.\d+)?)', filename).group(1))
    
    # Check if PS-PS exists
    is_ps_ps = 'PS-PS' in filename
    
    # Extract mchi
    mchi = float(re.search(r'mchi(\d+(?:\.\d+)?)', filename).group(1))
    
    # Extract gchi
    gchi = float(re.search(r'gchi(\d+(?:\.\d+)?)', filename).group(1))
    
    # Extract cHiggs
    c_higgs = float(re.search(r'cHiggs(\d+(?:\.\d+)?)', filename).group(1))
    
    return mphi, is_ps_ps, mchi, gchi, c_higgs


XenonEventRateFolder = "/Users/yxu/workbenches/XeArLoopworkbench/EventRatesMMA/Xenon_auto"
XenonEventRateFiles = sorted(os.listdir(XenonEventRateFolder))
LZExposure = 15.3
LZSR1g1 = 0.114
LZSR1g2 = 47.1

mphi_table = []
is_ps_ps_table = []
mchi_table = []
gchi_table = []
c_higgs_table = []
significance_table = []
significance_table_unit_exposure = []

for XenonEventRateFile in XenonEventRateFiles:
    mphi, is_ps_ps, mchi, gchi, c_higgs = parse_xenon_filename_auto(XenonEventRateFile)
    significance_current = findXenonModelSignificance(f"{XenonEventRateFolder}/{XenonEventRateFile}", 
                                                              f"/Users/yxu/workbenches/XeArLoopworkbench/LZSR1_bkg_total.txt", 
                                                              LZSR1g1, LZSR1g2, LZExposure)    
    significance_current_unit_exposure = findXenonModelSignificance(f"{XenonEventRateFolder}/{XenonEventRateFile}", 
                                                              f"/Users/yxu/workbenches/XeArLoopworkbench/LZSR1_bkg_total.txt", 
                                                              LZSR1g1, LZSR1g2, 1)   
    mphi_table.append(mphi)
    is_ps_ps_table.append(is_ps_ps)
    mchi_table.append(mchi)
    gchi_table.append(gchi)
    c_higgs_table.append(c_higgs)
    significance_table.append(significance_current)
    significance_table_unit_exposure.append(significance_current_unit_exposure)


In [11]:
# Create a pandas DataFrame with the results
import pandas as pd

results_df = pd.DataFrame({
    'mphi[GeV]': mphi_table,
    'is_ps_ps': is_ps_ps_table, 
    'mchi[GeV]': mchi_table,
    'gchi': gchi_table,
    'c_higgs': c_higgs_table,
    'significance_full_exposure': significance_table,
    'significance_unit_exposure [1/tonne^(0.5)/year^(0.5)]': significance_table_unit_exposure
})

# Save to CSV file
results_df.to_csv('xenon_auto_results_unit_exposure.csv', index=False)


In [17]:
def parse_xenon_filename(filename):
    """
    Parse Xenon filename to extract model parameters
    Format example: XeNNTable.txt, XeNNlTable.txt, XeNNpTable.txt, XeNNzTable.txt
    where NN is DM mass number
    
    Returns:
        dm_mass: DM mass in GeV
        med_mass: Mediator mass (7 GeV if 'Light' in name, else 15 GeV) 
        interaction: Interaction type ('S-PS' if 'p' in name, 'PS-PS' if 'z' in name)
        has_higgs: Boolean indicating if model has Higgs coupling ('l' before Table)
        isCombined: Boolean indicating if filename contains a second number
    """
    # Extract DM mass - find number after Xe
    import re
    dm_mass = float(re.search(r'Xe(\d+)', filename).group(1))
    
    # Check if there's a second number in the filename
    all_numbers = re.findall(r'\d+', filename)
    isCombined = len(all_numbers) > 1
    
    # Determine mediator mass
    med_mass = 7 if 'Light' in filename else 15
    
    # Determine interaction type
    if 'p' in filename:
        interaction = 'PS-PS'
    elif 'z' in filename:
        interaction = 'S-PS'
    else:
        interaction = None
        
    # Check for Higgs coupling
    has_higgs = bool(re.search(r'l(?=Table)', filename))
    
    return dm_mass, med_mass, interaction, has_higgs, isCombined

# Test the parser
dmMassTable = []
medMassTable = []
interactionTable = []
hasHiggsTable = []
isCombinedTable = []
for filename, signalSignificance in zip(XeEventRateFiles, xenonSignificanceTable):
    dm_mass, med_mass, interaction, has_higgs, isCombined = parse_xenon_filename(filename)
    dmMassTable.append(dm_mass)
    medMassTable.append(med_mass)
    interactionTable.append(interaction)
    hasHiggsTable.append(has_higgs)
    isCombinedTable.append(isCombined)


# Create pretty tables to display data grouped by interaction type
from prettytable import PrettyTable
import math

# Create table for S-PS interaction
table_sps = PrettyTable()
table_sps.field_names = ["DM Mass (GeV)", "Mediator Mass (GeV)", "Higgs Coupling", "log10(Significance)"]
table_sps.title = "LXe PS-PS Interaction, gSM = 0.6, gDM = 0.7"

# Create table for PS-PS interaction 
table_psps = PrettyTable()
table_psps.field_names = ["DM Mass (GeV)", "Mediator Mass (GeV)", "Higgs Coupling", "log10(Significance)"]
table_psps.title = "LXe S-PS Interaction, gSM = 0.6, gDM = 0.7"

# Sort data into appropriate tables
for dm, med, inter, higgs, sig, combined in zip(dmMassTable, medMassTable, interactionTable, hasHiggsTable, xenonSignificanceTable, isCombinedTable):
    if not combined:
        if inter == 'PS-PS':
            table_sps.add_row([dm, med, higgs, f"{math.log10(sig):.2f}"])
        elif inter == 'S-PS':
            table_psps.add_row([dm, med, higgs, f"{math.log10(sig):.2f}"])

print(table_sps)
print("\n")
print(table_psps)


+----------------------------------------------------------------------------+
|                LXe PS-PS Interaction, gSM = 0.6, gDM = 0.7                 |
+---------------+---------------------+----------------+---------------------+
| DM Mass (GeV) | Mediator Mass (GeV) | Higgs Coupling | log10(Significance) |
+---------------+---------------------+----------------+---------------------+
|      10.0     |          7          |     False      |        -2.51        |
|      10.0     |          7          |      True      |        -2.28        |
|      10.0     |          15         |     False      |        -3.37        |
|      10.0     |          15         |      True      |        -3.13        |
|     200.0     |          7          |     False      |        -2.27        |
|     200.0     |          7          |      True      |        -2.27        |
|     200.0     |          15         |     False      |        -2.56        |
|     200.0     |          15         |      True   